__DAT341 - Assignment 5 - Group 69__

This notebook is formatted as such :

- First, all the packages are imported and the data is loaded (first cell)
- Then, the utility functions are defined such as :
    - Training function for our models
    - Evaluation function
    - Blind test set prediction function
- Finally, differents models and techniques will be tried in the following cells.

**1 - Preparation**

In [1]:
# Basic packages
import os
import sys
import time
import numpy as np
from tqdm import tqdm
from contextlib import contextmanager

# PyTorch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# Torchvision packages
import torchvision
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision import models

# Importing data
train_dir = 'a5_data/train'
val_dir = 'a5_data/val'
test_dir = 'a5_data/test_blind'

torch.cuda.empty_cache()
# Transformations, normalization and augmentation
transform_basic = v2.Compose([
    v2.ToImage(),
    v2.Resize((224, 224)),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    #v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No data augmentation
train_dataset_basic = ImageFolder(train_dir, transform=transform_basic)
val_dataset_basic = ImageFolder(val_dir, transform=transform_basic)
test_dataset_basic = ImageFolder(test_dir, transform=transform_basic)

train_loader_basic = DataLoader(train_dataset_basic, batch_size=32, shuffle=True)
val_loader_basic = DataLoader(val_dataset_basic, batch_size=32, shuffle=False)
test_loader_basic = DataLoader(test_dataset_basic, batch_size=32, shuffle=False)
loader_basic = [train_loader_basic, val_loader_basic, test_loader_basic]

# Data augmentation
train_dataset = ImageFolder(train_dir, transform=transform)
val_dataset = ImageFolder(val_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
loader = [train_loader, val_loader, test_loader]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**2 - Utility functions**

In [3]:
class Classifier():
    def __init__(self,model,criterion,optimizer,loader,features_location=None):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.features_location = features_location
        self.loader= loader

        if self.features_location is not None:
            self.train_features = torch.load(f'{self.features_location}/train.pt').to(device)
            self.val_features = torch.load(f'{self.features_location}/val.pt').to(device)
            self.test_features = torch.load(f'{self.features_location}/test.pt').to(device)
            
        
    # To evaluate our model on the validation set
    def validate(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for i,(images, labels) in tqdm(enumerate(self.loader[1])):
                images, labels = images.to(device), labels.to(device)
                if self.features_location is not None:
                    images = self.val_features[i*32:(i*32 + labels.size(0)),:].to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy on validation set: {100 * correct / total}%')
    
    def predict(self,name):
        if not os.path.exists('predictions'):
            os.makedirs('predictions')
        predictions = []
        with torch.no_grad():
            for i,(images, labels) in tqdm(enumerate(self.loader[2])):
                images, labels = images.to(device), labels.to(device)
                if self.features_location is not None:
                    images = self.test_features[i*32:(i*32 + labels.size(0)),:].to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                predictions += ['MEL' if label == 0 else 'NV' for label in predicted]

            # Save the predictions to a file
            with open(f'predictions/{name}.txt', 'w') as f:
                for item in predictions:
                    f.write("%s\n" % item)
                    
    def train(self):
        start_time = time.time()
        # Training loop
        num_epochs = 10
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            for i,(images, labels) in tqdm(enumerate(self.loader[0]), total=len(self.loader[0])):
                images, labels = images.to(device), labels.to(device)
                if self.features_location is not None:
                    with torch.no_grad():
                        images = self.train_features[i*32:(i*32 + labels.size(0)),:].to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images).to(device)
                loss = self.criterion(outputs, labels)
                
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            print(f'Epoch {epoch+1}, Loss: {running_loss/len(self.loader[0])}')

        print(f'Training finished - Elapsed time: {time.time() - start_time} s.')

**3 - Models**

1) Simple CNN - No fancy feature extractor or anything

In [8]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 53 * 53, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 53 * 53)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
    
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

simple_CNN_basic = Classifier(model, criterion, optimizer, loader_basic)
simple_CNN_basic.train()
simple_CNN_basic.validate()
simple_CNN_basic.predict('simpleCNN_basic')

100%|██████████| 201/201 [00:11<00:00, 17.28it/s]


Epoch 1, Loss: 0.5265359819231935


100%|██████████| 201/201 [00:11<00:00, 17.34it/s]


Epoch 2, Loss: 0.4438179159787164


100%|██████████| 201/201 [00:11<00:00, 16.98it/s]


...

100%|██████████| 201/201 [00:11<00:00, 17.88it/s]


Epoch 10, Loss: 0.33564173797173286
Training finished - Elapsed time: 117.14580607414246 s.


40it [00:01, 23.53it/s]


Accuracy on validation set: 78.35463258785943%


43it [00:02, 20.32it/s]


In [19]:
simple_CNN= Classifier(model, criterion, optimizer, loader)
simple_CNN.train()
simple_CNN.validate()
simple_CNN.predict('simpleCNN')

100%|██████████| 201/201 [00:15<00:00, 13.04it/s]


Epoch 1, Loss: 0.37341942130333156


100%|██████████| 201/201 [00:15<00:00, 13.07it/s]


Epoch 2, Loss: 0.36089391917435093


100%|██████████| 201/201 [00:15<00:00, 13.13it/s]


...

100%|██████████| 201/201 [00:15<00:00, 12.84it/s]


Epoch 10, Loss: 0.31357101973757817
Training finished - Elapsed time: 154.18199253082275 s.


40it [00:02, 17.04it/s]


Accuracy on validation set: 80.5111821086262%


43it [00:02, 14.54it/s]


1.2 Batch normalization



In [20]:
class BatchNormCNN(nn.Module):
  def __init__(self):
    super(BatchNormCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.bn1 = nn.BatchNorm2d(6)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.bn2 = nn.BatchNorm2d(16)
    self.fc1 = nn.Linear(16 * 53 * 53, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.bn1(self.conv1(x))))
    x = self.pool(F.relu(self.bn2(self.conv2(x))))
    x = x.view(-1, 16 * 53 * 53)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
    
model = BatchNormCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Batchnorm_CNN = Classifier(model, criterion, optimizer,loader)
Batchnorm_CNN.train()
Batchnorm_CNN.validate()
Batchnorm_CNN.predict('BatchNormCNN')

100%|██████████| 201/201 [00:17<00:00, 11.40it/s]


Epoch 1, Loss: 0.4498662324390601


100%|██████████| 201/201 [00:17<00:00, 11.29it/s]


Epoch 2, Loss: 0.3964048334911688


100%|██████████| 201/201 [00:16<00:00, 12.53it/s]


...

100%|██████████| 201/201 [00:15<00:00, 12.80it/s]


Epoch 10, Loss: 0.3069242251601385
Training finished - Elapsed time: 160.86593914031982 s.


40it [00:02, 16.56it/s]


Accuracy on validation set: 82.50798722044729%


43it [00:02, 14.64it/s]


1.3 - Residual Connections

Here we only do a small one, after we'll use resnet that uses deep residual connections

In [21]:
class BatchNorm_ResidualCNN(nn.Module):
  def __init__(self):
    super(BatchNorm_ResidualCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.bn1 = nn.BatchNorm2d(6)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.bn2 = nn.BatchNorm2d(16)
    self.fc1 = nn.Linear(16 * 53 * 53, 120)
    self.fc2 = nn.Linear(120, 120)
    self.fc3 = nn.Linear(120, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.bn1(self.conv1(x))))
    x = self.pool(F.relu(self.bn2(self.conv2(x))))
    x = x.view(-1, 16 * 53 * 53)
    
    x = F.relu(self.fc1(x))
    residual = x
    x = F.relu(self.fc2(x) + residual)
    x = self.fc3(x)
    return x
    
model = BatchNorm_ResidualCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Batchnorm_ResidualCNN = Classifier(model, criterion, optimizer,loader)
Batchnorm_ResidualCNN.train()
Batchnorm_ResidualCNN.validate()
Batchnorm_ResidualCNN.predict('BatchNorm_ResidualCNN')

100%|██████████| 201/201 [00:16<00:00, 12.47it/s]


Epoch 1, Loss: 0.451251354383592


100%|██████████| 201/201 [00:16<00:00, 12.26it/s]


Epoch 2, Loss: 0.396692874123208


100%|██████████| 201/201 [00:15<00:00, 12.60it/s]


...

100%|██████████| 201/201 [00:15<00:00, 12.61it/s]


Epoch 10, Loss: 0.31321633466292376
Training finished - Elapsed time: 158.53012371063232 s.


40it [00:02, 17.07it/s]


Accuracy on validation set: 83.86581469648563%


43it [00:02, 14.47it/s]


2 - ResNet

In [22]:
# Load a pre-trained resnet18 model
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze it
for param in resnet.parameters():
    param.requires_grad = False
    
# Modify the last layer of ResNet to match the input size of the first linear layer
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Identity()

# Define the sequential model with ResNet and additional linear layers
model = nn.Sequential(
    resnet,
    nn.Linear(num_ftrs, 512),   # First linear layer
    nn.ReLU(),                  # ReLU activation
    nn.Linear(512, 256),        # Second linear layer
    nn.ReLU(),                  # ReLU activation
    nn.Linear(256, 2)           # Output layer with output size of 2
)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Classifier2 = Classifier(model, criterion, optimizer,loader)
Classifier2.train()
Classifier2.validate()
Classifier2.predict('Resnet18')


100%|██████████| 201/201 [00:15<00:00, 12.70it/s]


Epoch 1, Loss: 0.5371244515649122


100%|██████████| 201/201 [00:15<00:00, 12.93it/s]


Epoch 2, Loss: 0.3796119381539264


100%|██████████| 201/201 [00:15<00:00, 12.95it/s]


...

100%|██████████| 201/201 [00:15<00:00, 12.98it/s]


Epoch 10, Loss: 0.3098829149532674
Training finished - Elapsed time: 156.21048378944397 s.


40it [00:03, 12.97it/s]


Accuracy on validation set: 87.30031948881789%


43it [00:03, 12.81it/s]


3 - VGG19

In [4]:
# get the "features" portion of VGG19 (we will not need the "classifier" portion)
vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features
vgg.eval()

# freeze all parameters
for param in vgg.parameters():
    param.requires_grad_(False)
    
# Define the sequential model with VGG and additional linear layers
model = nn.Sequential(
    vgg,
    nn.Flatten(),               
    nn.Linear(25088, 4096),     
    nn.ReLU(),                  
    nn.Dropout(0.5),            
    nn.Linear(4096, 120),      
    nn.ReLU(),                 
    nn.Dropout(0.5),            
    nn.Linear(120, 2)         
)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
Classifier3 = Classifier(model, criterion, optimizer,loader)
Classifier3.train()
Classifier3.validate()
Classifier3.predict('VGG19')

100%|██████████| 201/201 [00:52<00:00,  3.83it/s]


Epoch 1, Loss: 0.43497797327848214


100%|██████████| 201/201 [00:44<00:00,  4.55it/s]


Epoch 2, Loss: 0.32485222571821354


100%|██████████| 201/201 [00:44<00:00,  4.54it/s]


...

100%|██████████| 201/201 [00:44<00:00,  4.56it/s]


Epoch 10, Loss: 0.1862778169895286
Training finished - Elapsed time: 455.8945097923279 s.


40it [00:08,  4.94it/s]


Accuracy on validation set: 85.78274760383387%


43it [00:08,  4.85it/s]


4 - State of the art : DINOv2

In [6]:
# To skip the message 'No module triton detected' on Windows
#######################
@contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

with suppress_stdout_stderr():
    dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').cuda()
#######################

# freeze all parameters
for param in dinov2.parameters():
    param.requires_grad_(False)

dinov2.eval()
# Define the sequential model with VGG and additional linear layers
model = nn.Sequential(
    dinov2,
    nn.Linear(384, 512),   # First linear layer
    nn.ReLU(),                  # ReLU activation
    nn.Linear(512, 256),        # Second linear layer
    nn.ReLU(),                  # ReLU activation
    nn.Linear(256, 2)           # Output layer with output size of 2
)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Classifier4 = Classifier(model, criterion, optimizer,loader)
Classifier4.train()
Classifier4.validate()
Classifier4.predict('Dinov2')

100%|██████████| 201/201 [00:25<00:00,  7.79it/s]


Epoch 1, Loss: 0.41266336547794624


100%|██████████| 201/201 [00:25<00:00,  7.88it/s]


Epoch 2, Loss: 0.31328935498621924


100%|██████████| 201/201 [00:27<00:00,  7.35it/s]


...

100%|██████████| 201/201 [00:25<00:00,  7.84it/s]


Epoch 10, Loss: 0.21663121204471114
Training finished - Elapsed time: 263.1771514415741 s.


40it [00:05,  7.90it/s]


Accuracy on validation set: 88.25878594249201%


43it [00:05,  7.76it/s]


Let's now try to save vgg19 features and then loading them in the training process

In [4]:
def get_features(loader,name,model,model_name):
    if os.path.exists(f'.features/{model_name}/{name}.pt'):
        return torch.load(f'.features/{model_name}/{name}.pt')
    
    features = []
    for batch, _ in tqdm(loader, desc=f'Extracting features from {name}'):
        batch = batch.to(device)
        # Extract the features
        with torch.no_grad():
            feature_batch = model(batch)
        features.append(feature_batch)
    features = torch.cat(features)
    if not os.path.exists('.features'):
        os.makedirs('.features')
    if not os.path.exists(f'.features/{model_name}'):
        os.makedirs(f'.features/{model_name}')
    torch.save(features,f'.features/{model_name}/{name}.pt')
    return features

vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features
vgg.to(device)

train_features = get_features(train_loader,'train',vgg,'vgg')
val_features = get_features(val_loader,'val',vgg,'vgg')
test_features = get_features(test_loader,'test',vgg,'vgg')

class Classifier_basic(nn.Module):
    def __init__(self):
        super(Classifier_basic, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(25088, 4096)
        self.bn1 = nn.BatchNorm1d(4096)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 120)
        self.bn2 = nn.BatchNorm1d(120)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(120, 2)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
    
model = Classifier_basic().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
Classifier5 = Classifier(model, criterion, optimizer,loader,'.features/vgg')
Classifier5.train()
Classifier5.validate()
Classifier5.predict('VGG19_saved_features')

100%|██████████| 201/201 [00:21<00:00,  9.27it/s]


Epoch 1, Loss: 0.7338845931475435


100%|██████████| 201/201 [00:17<00:00, 11.34it/s]


Epoch 2, Loss: 0.7071934461000546


100%|██████████| 201/201 [00:17<00:00, 11.42it/s]


...

100%|██████████| 201/201 [00:17<00:00, 11.31it/s]


Epoch 10, Loss: 0.6937889596123007
Training finished - Elapsed time: 181.89839506149292 s.


40it [00:02, 17.72it/s]


Accuracy on validation set: 50.0%


43it [00:02, 14.78it/s]


Now, the loss won't go down. We tried different techniques but could not understand why. 

The running loss stays really low, which means the classifier model believes it has reached a local minima. 
We didn't succeed to identify why this happened. Below is another try with a simplified classifier and another implementation, that yields the same results

Actually this always answer NV and we can't comprehend why.

In [14]:
# To skip the message 'No module triton detected' on Windows
#######################
@contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

with suppress_stdout_stderr():
    dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').cuda()
#######################

# freeze all parameters
for param in dinov2.parameters():
    param.requires_grad_(False)

dinov2.eval()

model = nn.Sequential(
    nn.Linear(384,2)
)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

with torch.no_grad( ):
            
    features = []
    for batch, _ in tqdm(train_loader, desc=f'Extracting features from train'):
        batch = batch.to(device)
        # Extract the features
        with torch.no_grad():
            result = dinov2(batch)
        features.append(result)
        torch.cuda.empty_cache()


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i,(images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()
            images = features[i].to(device)
            labels = labels.to(device)
            outputs = model(images).to(device)
            loss = criterion(outputs, labels)   
            loss.backward(retain_graph=True)
            optimizer.step()
            
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')



100%|██████████| 201/201 [00:10<00:00, 19.91it/s]


Epoch 1, Loss: 0.765381932258606


100%|██████████| 201/201 [00:10<00:00, 19.85it/s]


Epoch 2, Loss: 0.7222750186920166


100%|██████████| 201/201 [00:10<00:00, 19.37it/s]


...

Epoch 8, Loss: 0.7762109041213989


100%|██████████| 201/201 [00:10<00:00, 19.94it/s]


Epoch 9, Loss: 0.7687806487083435


100%|██████████| 201/201 [00:10<00:00, 20.00it/s]

Epoch 10, Loss: 0.6908783316612244
